## yelp review data collection

In [ ]:
# import packaegs for webscraping reviews and sentiment analysis
import requests
from bs4 import BeautifulSoup
from yelpapi import YelpAPI
import pandas as pd
import numpy as np
import json
import ntpath
import os
from tqdm import tqdm
import re

In [ ]:
# initial setup
# open client id and api key
f = open(##your client id file location, "r")
content = f.readlines()
client_id = content[1]
api_key = content[4]
API_HOST = 'https://api.yelp.com' #The API url header
SEARCH_PATH = '/v3/businesses/search' #The path for an API request to find businesses
BUSINESS_PATH = '/v3/businesses/'  # The path to get data for a single business
REVIEW_PATH = '/v3/businesses/{id}/reviews' # The path to get reviews for a single business id 


In [ ]:
# turn data from Yelp API to dataframe
def get_df(data):
    b_id = []
    name = []
    review_count = []
    rating = []
    city = []
    address = []
    zip_code = []
    state = []
    b_type = []
    for i in range(len(data)):
        b_id.append(data[i]['id'])
        name.append(data[i]['name'])
        review_count.append(data[i]['review_count'])
        rating.append(data[i]['rating'])
        city.append(data[i]['location']['city'])
        address.append(data[i]['location']['address1'])
        zip_code.append(data[i]['location']['zip_code'])
        state.append(data[i]['location']['state'])
    meetfresh_data = {
                      'id': b_id,
                      'name': name,
                      'state': state, 
                      'city': city,
                      'zip code': zip_code,
                      'review count': review_count,
                      'rating': rating,
                      'address': address
                     }
    return pd.DataFrame(meetfresh_data)

In [ ]:
# turn meetfresh data to dataframe
def meetfresh_df():
    # get all the meet fresh locations in US 
    locations = []
    with open('meet_fresh_location.txt') as location_f:
        for line in location_f:
            locations.append(line.strip('\n'))
    # getting meet fresh data from Yelp API and turn into dataframe for futhur exploratory data analysis
    term = 'Meet Fresh'
    b_data =[]
    frames = []

    for i in range(len(locations)):
        b_data.append(yelp_api.search_query(term = term, location = location[i], limit = 1))
        frames.append(get_df(b_data[i]['businesses']))
    result = pd.concat(frames)
    return result

In [ ]:
# get single review of each business id
def get_single_review(api_key,business_id):
    import json
    import requests
    business_path = BUSINESS_PATH + business_id+"/reviews"
    url = API_HOST + business_path

    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }


    response = requests.request('GET', url, headers=headers).json()
    review_text = ''
    for review in response['reviews']:
        review_text += review['text']
    return review_text

In [ ]:
# get 3 comments for each Meet fresh locations
def get_reviews(id_list):

    restaurants_id = id_list

    if not restaurants_id:
        return None
    review_list = list()
    for r_id in restaurants_id:
        review_text = get_single_review(api_key, r_id)
        review_list.append(review_text)
    return review_list

## Google review data collection

In [ ]:
#输入店铺的地址
adress = "140 Brighton Ave, Boston, MA 02134美国"
#店铺评论的数量，100以下为1，200以下为2，以此类推
number = 1
#HXR请求地址头
url_header = "https://www.google.com/maps/preview/review/listentitiesreviews?authuser=0&hl=zh-CN&gl=fr&pb=!1m2!1y9935918602392389647!2y14807764379085832903"


In [ ]:
def get_single_location_review(adress, number, url_header):
    
    #初始化dataframe
    columns = ["username", "time", "note", "comment","lang"]
    comments = pd.DataFrame(columns = columns )
    #抓取评论并存入DataFrame
    for j in range(number):
    #地址
    url = url_header + '!2m2!1i'+str(j)+'00!2i100!3e1!4m5!3b1!4b1!5b1!6b1!7b1!5m2!1s7pFNYvTMGs2saYiEoZgP!7e81'
    #发送get请求
    text = requests.get(url).text
    #取代特殊字元
    pretext = ')]}\''
    text = text.replace(pretext,'')
    #把字串改为读取json
    soup = json.loads(text)
    #取出包含留言得list 。
    conlist = soup[2]
    #逐单抓取
    for i in conlist:
        comments = comments.append(pd.DataFrame({"username":[str(i[0][1])]
                    ,"time":[str(i[1])]
                     ,"note":[str(i[4])]
                     ,"comment":[str(i[3])]
                     ,"lang":[str(i[32])]
                     }))
    comments = comments.reset_index()
    #储存到本地为json
    return comments.to_json(adress+".json")
    

In [ ]:
def combine_all_review():
    #path = "/content/drive/MyDrive/Meet Fresh Project/Data/Jingxuan_Comments Google Map/"
    #files= os.listdir(path)
    columns = ["username", "time", "note", "comment","lang", "adresse"]
    comments = pd.DataFrame(columns = columns)
    for file in files:
        res = pd.read_json(path+ntpath.basename(file))
        comments = comments.append(res,ignore_index=True)

    #去掉多余的项目
    comments = comments.drop(columns = ["index", "level_0"])
    def clear_characters(text):
	return re.sub('美国.json', '', text)
    test["adresse"] = test["adresse"].apply(clear_characters)
    
    return comments.to_json("all_comments.json")

## Competitors Yelp data collection

In [ ]:
DATA_PATH = '../data/'
REVIEW_PATH = '../data/Segmented_data_review_Yelp/'

In [ ]:
def get_competitor():
  frames = []
  start = 26000
  end = start+1000
  for i in tqdm(range(start, end)):

  #location = locations.iloc[i]['city'] + " " + locations.iloc[i]['county_name']
    latitude = locations.iloc[i]['lat']
    longitude = locations.iloc[i]['lng']
    terms = ['Shaved Ice', "Bubble Tea"]

    for term in terms:
      offset = 0
      while 50 + offset < 1000:
          data = yelp_api.search_query(term = term, 
                                    #location = location,
                                      latitude = latitude,
                                      longitude = longitude,
                                      limit = 50, 
                                      offset = offset,
                                      categories ="bubbletea,beverage_stores,desserts,restaurants,shavedice,juicebars,coffee")
          frames.append(get_df(data['businesses']))
          if len(data['businesses']) < 50:
            break
          else:
            offset += 50
    result = pd.concat(frames)
    result = result.drop_duplicates()
    return result.to_csv(REVIEW_PATH + "competitor_list_" + str(start) + "-" + str(i) + ".csv")

In [ ]:
def combien_segmented_yelp_reviews():
  columns = ['id', 'name', 'state', 'city', 'zip code', 'review count', 'rating', 'address', 'categories', 'latitude', 'longitude']
  all_resto = pd.DataFrame(columns = columns)
  for file in REVIEW_PATH:
    res = pd.read_csv(path+ntpath.basename(file))
    all_resto = all_resto.append(res,ignore_index=True)

  all_resto = all_resto.drop_duplicates()
  all_resto.to_csv(DATA_PATH + "competitor_list_BubbleTea_ShavedIce.csv")